In [1]:
import pandas as pd
import json
import re

In [ ]:
run_name = "union"

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))


        # load JSONL
records_test = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_test.tsv") as f:
    for line in f:
        records_test.append(json.loads(line))

In [3]:
df = pd.DataFrame(records)
df.head()

,left,right,match,match_confidence
0,COL make VAL PIPER COL model VAL L21 COL serie...,COL make VAL PIPER COL model VAL L-21,0,0.990064
1,COL make VAL BOEING COL model VAL 737 COL seri...,COL make VAL BOEING COL model VAL 737-758,0,0.996390
2,COL make VAL SWEARINGEN COL model VAL SA227 CO...,COL make VAL CONSTRUCCIONES AERONAUTICAS SA CO...,0,0.999122
3,COL make VAL WACO COL model VAL DSO COL series...,COL make VAL CESSNA COL model VAL TP206A,0,0.999945
4,COL make VAL BELL COL model VAL UH1 COL series...,COL make VAL BELL COL model VAL UH-1F,1,0.691420


In [4]:


gold = pd.read_csv("../data/aircraft_er/" + run_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])
print(gold.head())

                                                left  \
0  COL make VAL PIPER COL model VAL L21 COL serie...   
1  COL make VAL BOEING COL model VAL 737 COL seri...   
2  COL make VAL SWEARINGEN COL model VAL SA227 CO...   
3  COL make VAL WACO COL model VAL DSO COL series...   
4  COL make VAL BELL COL model VAL UH1 COL series...   

                                               right  gold  
0              COL make VAL PIPER COL model VAL L-21     0  
1          COL make VAL BOEING COL model VAL 737-758     0  
2  COL make VAL CONSTRUCCIONES AERONAUTICAS SA CO...     0  
3           COL make VAL CESSNA COL model VAL TP206A     0  
4              COL make VAL BELL COL model VAL UH-1F     1  


In [5]:
df["gold"] = gold["gold"]

In [6]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_true = df["gold"]
y_pred = df["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.9872052182639237

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      8949
           1       0.97      0.97      0.97      3009

    accuracy                           0.99     11958
   macro avg       0.98      0.98      0.98     11958
weighted avg       0.99      0.99      0.99     11958


Confusion matrix:
 [[8872   77]
 [  76 2933]]


In [7]:
errors = df[df["gold"] != df["match"]]
print(errors[["left","right","gold","match","match_confidence"]].head(20))

                                                   left  \
39    COL make VAL DOUGLAS COL model VAL R4D COL ser...   
195   COL make VAL BOMBARDIER COL model VAL BD500 CO...   
259   COL make VAL AIRBUS COL model VAL A321 COL ser...   
313   COL make VAL SOCATA COL model VAL RALLYE 235 C...   
379   COL make VAL BOEING COL model VAL 767 COL seri...   
663   COL make VAL CHAMPION COL model VAL 8GCBC COL ...   
764   COL make VAL CESSNA COL model VAL 525 COL seri...   
985   COL make VAL CESSNA COL model VAL 560 COL seri...   
1261  COL make VAL BELL COL model VAL UH1 COL series...   
1353  COL make VAL HAWKER COL model VAL HAWKER 800 C...   
1418  COL make VAL VERTOL COL model VAL CH47 COL ser...   
1520  COL make VAL HAWKER COL model VAL HAWKER 800 C...   
1749  COL make VAL BELL COL model VAL 206 COL series...   
1768  COL make VAL CESSNA COL model VAL 208 COL seri...   
1827  COL make VAL BRITTEN NORMAN COL model VAL BN2A...   
1899  COL make VAL CHAMPION COL model VAL 8GCBC COL ... 

In [8]:
errors

,left,right,match,match_confidence,gold
39,COL make VAL DOUGLAS COL model VAL R4D COL ser...,COL make VAL DOUGLAS COL model VAL DC-3S/R4D-8,0,0.567717,1
195,COL make VAL BOMBARDIER COL model VAL BD500 CO...,COL make VAL BOMBARDIER INC COL model VAL BD-7...,0,0.663130,1
259,COL make VAL AIRBUS COL model VAL A321 COL ser...,COL make VAL AIRBUS COL model VAL A321-271N,1,0.797338,0
313,COL make VAL SOCATA COL model VAL RALLYE 235 C...,COL make VAL S.O.C.A.T.A. COL model VAL MS RAL...,0,0.849895,1
379,COL make VAL BOEING COL model VAL 767 COL seri...,COL make VAL BOEING COL model VAL 767-300,1,0.803126,0
...,...,...,...,...,...
11759,COL make VAL DOUGLAS COL model VAL DC3C COL se...,COL make VAL DOUGLAS COL model VAL DC3C-S4C4G,0,0.911587,1
11853,COL make VAL PIPER COL model VAL PA46 COL seri...,COL make VAL PIPER COL model VAL PA-30,1,0.586544,0
11866,COL make VAL CALLAIR COL model VAL B1 COL seri...,COL make VAL AERO COMMANDER COL model VAL CALL...,0,0.998326,1
11886,COL make VAL MBB COL model VAL BO105 COL serie...,COL make VAL MESSERSCHMITT BOLKOW BLOHM COL mo...,0,0.772749,1


In [9]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [ ]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "gold": row["gold"],
        "predicted": row["match"],
        "confidence": row["match_confidence"]
    })

aligned = pd.DataFrame(parsed)

In [12]:
left.head()

AttributeError: 'dict' object has no attribute 'head'

In [11]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)